## Altmap Experiments
### Compare altmap to map eq using networkx

In [6]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator)
import numpy as np

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [24]:
# compute essential cost function values for a barbell network with
# given clique size
def ring_of_cliques_cost(clique_size = 3, number_of_cliques = 5, print_output=False):
    nc = clique_size
    c = number_of_cliques
    
    m = (nc*(nc-1) + 2) * c / 2.0 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the connecting nodes

    # initial cost
    J_init = c*(nc-1)*p0*np.log2(1.0-p0) + c*pc*np.log2(1.0-pc)
    
    # maximum independent sets cost
    p1 = c*p0; p2 = (c-1)*p0+pc
    J_ind = (nc-1)*p1*np.log2(1.0-p1) + c*p2*np.log2(1.0-p2)
    
    # ground truth cost
    pi = (nc-1)*p0+pc; pnoti=1.0-pi; pinoti=1.0/(2*m); pii = pi - pinoti
    J_true = -c*(pii*np.log2(pii/pi**2) + pinoti*np.log2(pinoti/(pi*pnoti)))
    
    if print_output:
        print (f"\nRing of {c} cliques with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
    
    return J_init, J_true, J_ind

num_cliques = 64
nc_max = 50
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    J_init_list[i], J_true_list[i], J_ind_list[i] = ring_of_cliques_cost(nc, number_of_cliques=num_cliques)


plt.close('all')
fig, ax = plt.subplots()
ax.set_title(f'Ring of {num_cliques} cliques - objective over clique size')
ax.plot(nc_list, -J_true_list, 'x--', lw=3, ms=10, label='Ground truth')
ax.plot(nc_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, -J_ind_list, 'o--', label='Maximum independet sets')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$')
ax.set_ylabel('Altmap objective $\mathcal{G}(m)$')
ax.legend()

In [85]:
num_cliques = 30
clique_size = 3
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# run community detection
# communities_found, num_communities_found = infomap(G, altmap=False)
# communities_found, num_communities_found = infomap(G, altmap=True)
communities_found, num_communities_found = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


OrderedDict([(1, 1), (2, 1), (3, 1), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 3), (17, 3), (18, 3), (19, 3), (20, 3), (21, 3), (22, 3), (23, 3), (24, 3), (25, 3), (26, 3), (27, 3), (28, 5), (29, 5), (30, 5), (31, 5), (32, 5), (33, 5), (34, 5), (35, 5), (36, 5), (37, 6), (38, 6), (39, 6), (40, 6), (41, 6), (42, 6), (43, 6), (44, 6), (45, 6), (46, 9), (47, 9), (48, 9), (49, 9), (50, 9), (51, 9), (52, 7), (53, 7), (54, 7), (55, 7), (56, 7), (57, 7), (58, 7), (59, 7), (60, 7), (61, 8), (62, 8), (63, 8), (64, 8), (65, 8), (66, 8), (67, 8), (68, 8), (69, 8), (70, 4), (71, 4), (72, 4), (73, 4), (74, 4), (75, 4), (76, 4), (77, 4), (78, 4), (79, 4), (80, 4), (81, 4), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1)])
We found 9 communities.


In [86]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)

/home/chri/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


In [78]:
# generate network
num_cliques = 3
clique_size = 4
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * clique_size
for c in range(2, num_cliques+1):
    labels += [c]*clique_size
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = list(range(1,clique_size+1)) * num_cliques
communities_ind = dict(zip(nodes_ids, labels))

plt.close('all')
fig, axs = plt.subplots(1,3)
fig.suptitle(f'Sample ring of cliques networks')

drawNetwork(G, communities_true, ax=axs[0])
axs[0].set_xlabel('Ground truth')
drawNetwork(G, communities_init, ax=axs[1])
axs[1].set_xlabel('Initial partition')
drawNetwork(G, communities_ind, ax=axs[2])
axs[2].set_xlabel('Maximum independent sets')

/home/chri/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


Text(0.5, 0, 'Maximum independent sets')